In [ ]:
import numpy as np 
import pandas as pd 
import rasterio
import rasterio.features
import rasterio.warp
import folium
from pyproj import transform, Proj, CRS

from FDSSC import models, utils
from matplotlib import pyplot as plt

In [ ]:
datadir = "data"
dir2name = utils.walkdir(datadir)

In [ ]:
utils.collate_images(dir2name.values())

In [ ]:
dataset = rasterio.open(dir2files.values()[0][0])

In [ ]:
# read the csv into a pandas data frame
OCC_df = pd.read_csv('miningSites/processedMS.csv', header=0, index_col=False)

OCC_df = OCC_df.loc[OCC_df["MAJOR_COMMODITY_CODE"] == "Au"]


In [ ]:
m = folium.Map(location=[-31.743944369555788, 138.29644592694788], tiles='OpenStreetMap', zoom_start=8)


tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, 
        control = True, opacity=0.7).add_to(m)
root = "data/stacked_images"
for files in list(dir2name.values())[0]:
        dataset = rasterio.open(files)
        image_loc = folium.vector_layers.Polygon([(y, x) for x, y in utils.findshape(dataset)[0]["coordinates"][0]]).add_to(m)
for i in OCC_df.index:
    folium.Marker([OCC_df.loc[i,'LATITUDE'], 
                    OCC_df.loc[i,'LONGITUDE']]).add_to(m)

m

In [ ]:
# creating function to read in portions of an image

def extract_cubic(dataset, coords, size=(9, 9), ):
    '''takes in a opened rasterio dataset, a list of tuples of coords [(lat, long),], size of the extracted cubics'''

     image = dataset.read()